In [33]:
!pip install transformers --quiet
!pip install sentencepiece --quiet

In [34]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [35]:
!ls 'drive/MyDrive/Kuliah/Semester 7/Pemrosesan Bahasa Alami/Tugas besar/dataset'

SQuAD-v1.1.csv


In [36]:
import torch
import transformers
from transformers import T5Tokenizer, T5ForConditionalGeneration, T5Config
import spacy
import pandas as pd

In [37]:
DIR = 'drive/MyDrive/Kuliah/Semester 7/Pemrosesan Bahasa Alami/Tugas besar/'
SEQ_LENGTH = 512
PRETRAINED_MODEL = 't5-base'

In [38]:
df = pd.read_csv(DIR + 'dataset/SQuAD-v1.1.csv')

In [39]:
df = df.dropna()
df.isnull().values.any()

False

In [40]:
dataset = df[533:633].drop(columns=['title', 'answer_start', 'answer_end', 'question']).reset_index()
dataset.head()

,index,context,answer
0,533,"On January 7, 2012, Beyoncé gave birth to her ...","January 7, 2012"
1,534,"On January 7, 2012, Beyoncé gave birth to her ...",Blue Ivy Carter
2,535,"On January 7, 2012, Beyoncé gave birth to her ...",Lenox Hill Hospital in New York.
3,536,"On January 7, 2012, Beyoncé gave birth to her ...",Revel Atlantic City's Ovation Hall
4,537,"On January 7, 2012, Beyoncé gave birth to her ...",four


In [41]:
# dataset.to_csv(DIR + 'datauji-Question-generator.csv', index=False)

In [42]:
tokenizer = T5Tokenizer.from_pretrained(PRETRAINED_MODEL)
tokenizer.add_special_tokens(
    {'additional_special_tokens': ['<answer>', '<context>']}
)

2

In [43]:
saved = torch.load(DIR + 'model/qg_pretrained_t5_model_trained.pth', map_location='cpu')
config = T5Config(decoder_start_token_id=tokenizer.pad_token_id)
model = T5ForConditionalGeneration(config).from_pretrained(PRETRAINED_MODEL)
model.resize_token_embeddings(len(tokenizer))
model.load_state_dict(saved['model_state_dict'])

<All keys matched successfully>

In [44]:
def inference(dataset):
  input_texts = '<answer> ' + dataset['answer'] + ' <context> ' + dataset['context']
  input_sequences = tokenizer(
            input_texts.tolist(), 
            padding='max_length', 
            max_length=SEQ_LENGTH,
            truncation=True,
            return_tensors="pt"
        )
  
  output_sequences = model.generate(
      input_ids=input_sequences['input_ids'],
      attention_mask=input_sequences['attention_mask']
  )

  questions = tokenizer.batch_decode(output_sequences, skip_special_tokens=True)
  
  return pd.DataFrame({
      'context': dataset['context'],
      'question': questions,
      'answer': dataset['answer']
  })


In [45]:
results = inference(dataset)

In [46]:
results.to_csv(DIR + 'hasiluji-Question-generator-T5.csv')